In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib notebook

In [ ]:
sea_level_deep_time = pd.read_excel("sealevel_1ma.xlsx")

In [ ]:
temp_65ma = pd.read_csv("temp_65ma.csv", delimiter=";")

In [ ]:
f, axs = plt.subplots(3, 3)
axs[0, 0].set_ylabel("Sea Level", size="small")
axs[1, 0].set_ylabel("CO$_{2}$ (ppm)", size="small")
axs[2, 0].set_ylabel("Global Mean Temp (K)", size="small")

In [ ]:
sea_level_deep_time.set_index("AGE").plot(ax=axs[0,1])